# NAVER NEWS API를 활용한 수집

In [70]:
import os
import sys
import urllib.request
import requests
from dotenv import load_dotenv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

In [83]:
keyword="블랙핑크"


load_dotenv()
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

encText = urllib.parse.quote(f"{keyword}")
url = "https://openapi.naver.com/v1/search/news?query=" + encText
response = requests.get(url,headers={"X-Naver-Client-Id":client_id ,"X-Naver-Client-Secret":client_secret})
rescode = response.status_code

if(rescode==200):
    response_body = response.json()
    link_list = [x['link'] for x in response_body['items'] if 'naver' in x['link']]
    driver =webdriver.Chrome()
    for link in link_list :
        driver.get(link)
        time.sleep(1)
        article = driver.find_elements(By.CLASS_NAME, "_article_content")
        text = [x.text for x in article]
        print(text)
    print(link_list)
    driver.close()
else:
    print("에러가 발생했어요 ㅠ^ㅠ \n Error Code:" + rescode)


['로제와 헤어 스타일리스트 헤수스 게레로. 사진 ㅣ로제 SNS\n블랙핑크 로제가 최근 사망한 헤어 스타일리스트 헤수스 게레로(Jesus Guerrero)를 애도했다.\n\n로제는 24일 인스타그램 스토리에 “내가 가장 필요할 때 찾아온 가장 사랑스러운 천사. 너를 너무 사랑해. 그리고 벌써 보고 싶어”라는 추모 글을 올린 후 헤수스 게레로와 함께 찍은 사진을 공유했다.\n\n헤수스 게레로는 카일리 제너, 제니퍼 로페즈, 케이티 페리 등의 담당 헤어스타일리스트 유명하다. NBC NEWS 등 현지 외신에 따르면 지난 23일 향년 34세로 갑작스레 세상을 떠났다.\n\n유족들은 SNS를 통해 “그의 죽음은 매우 갑작스럽고 예기치 않게 찾아왔다. 우리는 그를 ‘가장 밝은 빛’으로 기억하겠다”라는 글을 남겼다.\n\n로제는 지난해 12월 발매한 첫 정규앨범 ‘로지’ 작업 당시 헤수스 게레로와 협업한 것으로 알려졌다.\n\n로제는 지난해 브루노 마스와 협업한 ‘APT’로 글로벌 음악시장에서 돌풍을 일으켰다.\n\n‘APT’ 뮤직비디오는 유튜브 조회수 10억뷰를 달성했고 정규 1집 ‘rosie’는 빌보드 메인 앨범 차트 ‘빌보드 200’에 3위로 진입해 K팝 여성 아티스트 최고 기록을 갱신했다.\n\n로제는 올해 블랙핑크 월드투어에 참여한다. 오는 7월 5~6일 한국을 시작으로 미국, 프랑스, 이탈리아, 스페인, 영국, 일본에서 팬들을 만난다.']
['로제, 고 헤수스 레게로/로제 인스타그램\n\n[마이데일리 = 박로사 기자] 그룹 블랙핑크 로제가 헤어 스타일리스트 헤수스 게레로(Jesus Guerrero)를 애도했다.\n\n로제는 지난 24일 자신의 인스타그램에 헤수스 게레로와 함께 찍은 사진을 올리며 고인을 추모했다. 그는 "가장 필요할 때 나에게 와준 가장 다정한 천사. 정말 많이 사랑하고, 벌써 그리워"라고 적었다.\n\n헤수스 게레로는 지난 23일 향년 34세의 나이로 세상을 떠났다. 고인은 생전 카일리 제너, 제니퍼 로페즈, 케이티 페리 등 유명 연예인들의 헤